# Li2 Grokking: Densify/Beta-only Colab Notebook

This notebook is for the **densify + beta / beta_delay** follow-up only.
It is meant for cloud runtimes that reset frequently (Colab), so it always updates the repo checkout.

What it does:
- Runs a multi-seed densified ratio grid (resumable) via `multiseed_fill.py`
- Computes `beta` from `grok_epoch` via `analyze_beta_transition.py`
- Computes `beta_delay` from `grok_delay` via `analyze_grok_speed.py --use_delay --max_delta_r ...`

Notes:
- This is compute-heavy (many training runs). Start with **one M** if you want a fast check.
- Outputs are written under `experiments/li2_scaling_law/results/`.


## 0) Configure repo URL

Set your GitHub repo URL (or a fork) and branch.


In [ ]:
REPO_URL = "https://github.com/<OWNER>/<REPO>.git"  # TODO: replace
BRANCH = "main"

REPO_DIR = "F-I-T"
EXPERIMENT_DIR = f"{REPO_DIR}/experiments/li2_scaling_law"


## 1) Setup environment

Colab usually has PyTorch. We install lightweight deps used by analysis/plotting.


In [ ]:
!python -V
!pip -q install numpy matplotlib tqdm pyyaml

import sys
import subprocess

try:
    import torch  # noqa: F401
except Exception:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "torch"])
    import torch  # noqa: F401

print('torch', torch.__version__, 'cuda?', torch.cuda.is_available())


## 2) Clone/update repo

If the repo already exists, this cell will update it to the latest `BRANCH`.


In [ ]:
from pathlib import Path
import subprocess

if REPO_URL.startswith('https://github.com/<OWNER>'):
    raise SystemExit('Please set REPO_URL to your repo (or a fork) first.')

repo_path = Path(REPO_DIR)
if not repo_path.exists():
    subprocess.check_call(['git', 'clone', '--depth', '1', '--branch', BRANCH, REPO_URL, REPO_DIR])
else:
    print('Repo already exists:', REPO_DIR)
    subprocess.check_call(['git', '-C', REPO_DIR, 'fetch', 'origin', BRANCH])
    subprocess.check_call(['git', '-C', REPO_DIR, 'checkout', BRANCH])
    subprocess.check_call(['git', '-C', REPO_DIR, 'pull', '--ff-only'])

assert Path(EXPERIMENT_DIR).exists(), f"Missing {EXPERIMENT_DIR}"
%cd {EXPERIMENT_DIR}


In [ ]:
# Sanity check: prevent running an outdated checkout
from pathlib import Path
import subprocess

required = [
    'train.py',
    'multiseed_fill.py',
    'analyze_beta_transition.py',
    'analyze_grok_speed.py',
]
missing = [p for p in required if not Path(p).exists()]
if missing:
    raise SystemExit('Missing required files (repo checkout likely outdated): ' + ', '.join(missing))

head = subprocess.check_output(['git', '-C', REPO_DIR, 'rev-parse', '--short', 'HEAD']).decode().strip()
print('Git HEAD:', head)
print('analyze_grok_speed has --max_delta_r?', '--max_delta_r' in Path('analyze_grok_speed.py').read_text(encoding='utf-8', errors='replace'))


## 3) Densify + beta / beta_delay (multi-seed)

Edit `SPEC` to control which M values and ratios you run.

Default spec densifies around previously observed boundaries:
- M=23: r_crit ~ 0.575
- M=41: r_crit ~ 0.490
- M=59: r_crit ~ 0.450

Tip: start with a single M (e.g., only `59:...`) if you want a fast run.


In [ ]:
OUT_DIR = "results/beta_multiseed_colab_v2"
SEEDS = "42,123,456"

# At least 1 below-boundary + 5+ above-boundary points per M.
# You can remove M blocks to reduce compute.
SPEC = (
    "23:0.56,0.58,0.59,0.60,0.61,0.62;"
    "41:0.48,0.50,0.51,0.52,0.53,0.54;"
    "59:0.44,0.46,0.47,0.48,0.49,0.50"
)

!python multiseed_fill.py --spec "{SPEC}" --seeds {SEEDS} --output_dir {OUT_DIR}

!python analyze_beta_transition.py --results_dir {OUT_DIR} --output_dir {OUT_DIR}/analysis_beta --min_prob 1.0 --min_points 5

# beta_delay (near-boundary cap)
!python analyze_grok_speed.py --results_dir {OUT_DIR} --output_dir {OUT_DIR}/analysis_delay_near --min_prob 1.0 --min_points 3 --use_delay --max_delta_r 0.07

!ls -la {OUT_DIR}/analysis_beta || true
!ls -la {OUT_DIR}/analysis_delay_near || true


## 4) Download artifacts

Zip the `results/` directory and download.


In [ ]:
import shutil
from pathlib import Path

zip_path = Path('li2_results_colab.zip')
if zip_path.exists():
    zip_path.unlink()

shutil.make_archive(zip_path.with_suffix(''), 'zip', root_dir='.', base_dir='results')
print('Wrote', zip_path)

try:
    from google.colab import files
    files.download(str(zip_path))
except Exception as e:
    print('Download helper not available:', e)
